In [1]:
mapbox_access_token = 'pk.eyJ1IjoidmFzYXJoZWx5aW8iLCJhIjoiY2prYjV2djh0M2R3NDNxbWw3dTFqdGZvbyJ9.stZ2MjMsogAYJ9fMb-lrsg'

In [2]:
import geopandas as gdf
import pandas as pd
import plotly.graph_objs as go
import plotly as py

from plotly.graph_objs import *
from shapely.geometry import Point

import rpy2

In [3]:
path_to_df="../../src/models/sequence_analysis/data/clustering_results/"

In [4]:
file_name="cluster_results_Jul_Aug_10000_sample.csv"

In [5]:
df_clusters = pd.read_csv(path_to_df+file_name)

In [6]:
df_centroids=pd.read_csv("../../viz/comune_centroids.csv")

In [7]:
df_clusters.head()

,Unnamed: 0,cluster,customer_nr,locations,medoids
0,0,1,2171235,"1154, 1076, 1044, 47006, 48017, 47006, 47005, ...",0
1,1,2,7247821,"48017, 48036, 48035, 48036, 48035, 48036, 4803...",0
2,2,3,6970921,"46005, 46024, 46005, 46024, 46005, 46024, 46005",0
3,3,3,2086771,"48017, 48006, 100005, 47002, 47014, 47020, 470...",0
4,4,2,2409392,"104018, 49005, 49009",0


In [37]:
def get_centroids_trip(df_trip):
    """
    Merge controids of municipalities to DataFrame

    Parameters:
    trip: DataFrame with municipalities in a trip (column format)
    """

    trip_centroids = df_trip.merge(df_centroids,
                                how='inner',
                                left_on='pro_com',
                                right_on='pro_com')
    return trip_centroids

In [38]:
colors = ["rgb(255,255,0)","rgb(34,139,34)","rgb(220,20,60)","rgb(70,130,180)","rgb(128,0,128)"]

In [8]:
df_med = df_clusters[df_clusters['medoids'] == 1]

In [18]:
df_med.head()

,Unnamed: 0,cluster,customer_nr,locations,medoids
3026,3026,2,5955872,"27042, 27023, 28100, 28060, 28055, 29041, 2902...",1
6031,6031,5,1418535,"50029, 50022, 52032, 52006, 52032",1
7209,7209,3,1179935,"13075, 13154, 13135, 13036, 13227, 15182, 1517...",1
7645,7645,1,6830799,"1154, 1006, 1229, 1228, 1219, 1008, 1219, 1024...",1
9225,9225,4,6804043,"100007, 100001, 100006, 48017",1


In [35]:
list(map(int,df_med['locations'].iloc[0].split(', ')))

[27042,
 27023,
 28100,
 28060,
 28055,
 29041,
 29023,
 29037,
 29023,
 38008,
 37028,
 37052,
 37002,
 37006,
 37034,
 37040,
 48018,
 48017,
 48024,
 48044,
 48028,
 48014,
 50032,
 50009,
 50029,
 50008,
 50026,
 50008,
 50038,
 50029,
 50009,
 50033,
 48019,
 50032,
 48014,
 48043,
 48017,
 48036,
 48035,
 48052,
 51033,
 51039,
 51016,
 51002,
 51016,
 51002,
 51012,
 51017,
 52011,
 54012,
 55023,
 55002,
 55023,
 56018,
 56022,
 56042,
 57035,
 58018,
 58091]

In [27]:
list(map(int, df_med['locations'].iloc[0].tolist()[0].split(',')))

AttributeError: 'str' object has no attribute 'tolist'

In [57]:
df_clus = df_med[df_med['cluster'] == 1]
df_trajs = df_clusters[(df_clusters['medoids'] == 0) & (df_clusters['cluster'] == 1)][:n_trajectories_per_cluster-1]
df_trips = pd.concat([df_clus,df_trajs])
df_trips.head()

,Unnamed: 0,cluster,customer_nr,locations,medoids
7645,7645,1,6830799,"1154, 1006, 1229, 1228, 1219, 1008, 1219, 1024...",1
0,0,1,2171235,"1154, 1076, 1044, 47006, 48017, 47006, 47005, ...",0
8,8,1,2353132,"32006, 32005, 32006, 31003, 31016, 31021, 3102...",0
29,29,1,7408776,"52028, 52028, 52022, 52004, 52012, 52004, 5201...",0


In [58]:
clusters = df_clusters['cluster'].unique()
n_clusters = len(clusters)
# separate medoids
df_med = df_clusters[df_clusters['medoids'] == 1]
n_trajectories_per_cluster = 4

df_clus = df_med[df_med['cluster'] == c]
df_trajs = df_clusters[(df_clusters['medoids'] == 0) & (df_clusters['cluster'] == c)][:n_trajectories_per_cluster-1]
df_trips = pd.concat([df_clus,df_trajs])

# cluster loop
data=[]
for c in range(0,len(df_trips)):
    print(c)
    trip = list(map(int,df_trips['locations'].iloc[c].split(', ')))
    df_trip = pd.DataFrame(data={'pro_com': trip})
    # get centroids of each trip
    df_trip_centroids = get_centroids_trip(df_trip)
    steps = []
    for i in range(len(df_trip_centroids)):
        steps.append((df_trip_centroids['lon'][i],df_trip_centroids['lat'][i]))
#     print(steps)
    site_lat = list(df_trip_centroids.lon)
    site_lon = list(df_trip_centroids.lat)
    data.append(go.Scattermapbox(
        lat = site_lat,
        lon = site_lon,
        mode='markers',
        marker=dict(
            size=6,
            color=colors[c-1]
        )))
#plot_single_trajectory()

0
1
2
3


In [59]:
layout = go.Layout(
    title="trajectory points",
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=5,
        style='dark'
    ),
)

In [60]:
fig = dict(data=data, layout=layout)
py.offline.plot(fig, 'filename.html')

'file:///mnt/data/kmohan/TPT_tourism/dev/descriptives/temp-plot.html'

In [ ]:
steps

In [76]:
clusters = df_clusters['cluster'].unique()
n_clusters = len(clusters)
# separate medoids
df_med = df_clusters[df_clusters['medoids'] == 1]
n_trajectories_per_cluster = 4

df_clus = df_med[df_med['cluster'] == 2]
df_trajs = df_clusters[(df_clusters['medoids'] == 0) & (df_clusters['cluster'] == 2)][:n_trajectories_per_cluster-1]
df_trips = pd.concat([df_clus,df_trajs])

# cluster loop

data=[]
cluster_name = ["Trajectory 1","Trajectory 2","Trajectory 3","Trajectory 4"]
for c in range(0,len(df_trips)):
    print(c)
    trip = list(map(int,df_trips['locations'].iloc[c].split(', ')))
    df_trip = pd.DataFrame(data={'pro_com': trip})
    # get centroids of each trip
    df_trip_centroids = get_centroids_trip(df_trip)
    steps = []
    for i in range(len(df_trip_centroids)):
        steps.append((df_trip_centroids['lon'][i],df_trip_centroids['lat'][i]))
    data.append(go.Scattermapbox(
            lat=[item_x[0] for item_x in steps],
            lon=[item_y[1] for item_y in steps],
            mode='markers+lines',
#             text = cluster_name[c-1],
#             hoverinfo='text',
#             name=cluster_name[c-1],
            marker=dict(
                size=6,
                color=colors[c-1]
        ),
            line=dict(
                width=3,
)))

# data = [go.Scattermapbox(
#             lat=[item_x[0] for item_x in steps],
#             lon=[item_y[1] for item_y in steps],
#             mode='markers+lines',
#             marker=dict(
#             size=6
#             )
#         )
#     ]

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style='dark',
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=5
    ),
)

0
1
2
3


In [77]:
fig = dict(data=data, layout=layout)
py.offline.plot(fig, 'trajectory_cluster_example.html')

'file:///mnt/data/kmohan/TPT_tourism/dev/descriptives/temp-plot.html'

In [ ]:
lat=[item_x[0] for item_x in steps]
lon=[item_y[1] for item_y in steps]
lat
lon

In [ ]:
marker

In [ ]:
site_lat = list(df_trip_centroids.lon)
site_lon = list(df_trip_centroids.lat)
[site_lat[0:len(site_lat)-1],site_lat[1:len(site_lat)]]

In [ ]:
df_trip_centroids.lat

In [ ]:
for i in clusters:
    site_lat = x.avg_lat
    site_lon = x.avg_lon
    locations_name = x.index
    data.append(go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=3,
            color=colors[int(i)],
            opacity=0.8
        ),
        text=locations_name,
        hoverinfo='none',
        name=i) #names[int(i)])
    )

In [ ]:
def df_to_geojson(df, properties, lat='latitude', lng='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lng : the name of the column in the dataframe that contains longitude data
    """

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    # x is the equivalent of the row, df.iterrows converts the dataframe in to a pd.series object
    # the x is a counter and has no influence
    for x, row in df.iterrows():

        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'LineString',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [float(row[lng]),float(row[lat])]

        # be aware that the dataframe is a pd.series
        #print('rowitem converts to ndarray(numpy) :\n ', row)
        # convert the array to a pandas.serie
        geo_props = pd.Series(row)

        # for each column, get the value and add it as a new feature property
        # prop determines the list from the properties
        for prop in properties:

            #loop over the items to convert to string elements

            #convert to string
            if type(geo_props[prop]) == float:
                #print('ok')
                geo_props[prop] = str(int(geo_props[prop]))

            # now create a json format, here we have to make the dict properties
            feature['properties'][prop] = geo_props[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    return geojson

In [ ]:
geo_json_traj=df_to_geojson(df_trip_centroids, ['pro_com'], lat='lat', lng='lon')

In [ ]:
type(df_trip_centroids['lat'][0])

In [ ]:
colors

In [ ]:
import plotly as py
import plotly.graph_objs as go

In [ ]:
df_trip_centroids.lat[0]

In [ ]:
import plotly.graph_objs as go
data=[go.Scattermapbox(
    lat=list(df_trip_centroids['lat']),
    lon=list(df_trip_centroids.lon),
            mode='markers',
            marker=dict(
                size=8,
                color='rgb(255,165,0)'
            ),
            text=[str(i) for i in df_trip_centroids.pro_com],
            hoverinfo='none')]

     
layout = go.Layout(
title='Lines in mapbox, via geojson data',
autosize=True,
hovermode='closest',

mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=5,
        style='light'))

In [ ]:
fig = dict(data=data, layout=layout)
py.offline.plot(fig, 'filename.html')

In [ ]:
import plotly as py

In [ ]:
df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

In [ ]:
import plotly
plotly.tools.set_credentials_file(username='velf', api_key='5odqaBH9IzpJVnImX8Og')

In [ ]:
import plotly.plotly as py
import pandas as pd

"""df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()"""

airports = [ dict(
        type = 'scattergeo',
        lon = df_trip_centroids['lon'],
        lat = df_trip_centroids['lat'],
        hoverinfo = 'text',
        text = df_trip_centroids['pro_com'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]
        
flight_paths = []
for i in range( len( df_trip_centroids)-1 ):
    flight_paths.append(
        dict(
            type = 'scattergeo',
            lon = [ df_trip_centroids['lon'][i], df_trip_centroids['lon'][i+1] ],
            lat = [ df_trip_centroids['lat'][i], df_trip_centroids['lat'][i+1] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
        )
    )
    
layout = dict(
        title = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
        showlegend = False, 
        geo = dict(
            scope='usa',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=flight_paths + airports, layout=layout )
py.iplot( fig, filename='d3-flight-paths' )

In [ ]:
fig=dict(data+layout)
py.offline.plot(fig, filename='file.html')

In [ ]:
go.Line?

In [ ]:
fig = dict(data=data, layout=layout)
py.offline.plot(fig, filename='d3-flight-paths' )

In [ ]:
layout = [go.Layout(
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=7,
        style='light'
    ),
)]

In [ ]:
layout = go.Layout(
        title = 'Trajectory Meroids1',
        autosize=True,
        hovermode='closest',
        showlegend=False,
        mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=5,
        style='light'
    ),
)

    
#fig = dict( layers=trajectory, layout=layout )
#py.offline.plot(fig, filename='d3-flight-paths' )

In [ ]:
import sys
import os
sys.path.append("../../src/utils/")
from read_shapefiles import read_files

import pandas as pd
import plotly as py
import plotly.graph_objs as go
from shapely.geometry import Point
import geopandas as gpd



def read_tusc(path):
    """
    Helper function to read in the Tuscany shapefile
    """
    path_shapefiles, regions, provinces, territories, municipalities, crs = read_files.read_shapefile_data(path, 'shape_files_path.json')
    df_reg_tus = read_files.read_shapefiles_in(True, path_shapefiles, regions, crs)
    return df_reg_tus


def filter_tusc(result):
    """
    Filter geopoints to get only Tuscany
    """
    geometry = [Point(xy) for xy in zip(result['avg_lon'], result['avg_lat'])]
    df_reg_tus=read_tusc("../src/utils/read_shapefiles/")
    geo_cluster = gpd.GeoDataFrame(result, crs=df_reg_tus.crs, geometry=geometry)
    tusc_mask = geo_cluster.within(df_reg_tus.loc[8, 'geometry'])
    result_tusc = geo_cluster.loc[tusc_mask]
    return result_tusc

def create_file_name_and_path(country, season):
    path='../results/kmeans/'
    country_ = country.lower()
    season_ = season.replace('-','_')
    file_name=country_+"_"+season_
    newpath=path+file_name+'/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    return (newpath,file_name)

def create_data(result, names, colors):
    """
    Create data for visulaization
    """
    
    data=[]
    df=filter_tusc(result)
    clusters=df['label'].value_counts().index
    for i in clusters:
        x=df[df['label']==i][['avg_lat', 'avg_lon', 'label']]
        site_lat = x.avg_lat
        site_lon = x.avg_lon
        locations_name = x.index
        data.append(go.Scattermapbox(
            lat=site_lat,
            lon=site_lon,
            mode='markers',
            marker=dict(
                size=3,
                color=colors[int(i)],
                opacity=0.8
            ),
            text=locations_name,
            hoverinfo='none',
            name=names[int(i)])
        )
    return data

def plot_kmeans(result, names, colors, country, season, mapbox_access_token):
    if len(result)>100000:
        result=result.sample(frac=0.1, replace=True)
    d=create_data(result, names, colors)
    layout = go.Layout(
    title=country.title()+" in the "+season.title(),
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.45089368,
            lon=11.12636257
        ),
        pitch=0,
        zoom=7,
        style='light'
    ),
)

    fig = dict(data=d, layout=layout)
    newpath,filename=create_file_name_and_path(country, season)
    print(newpath,filename)
    py.offline.plot(fig, filename=newpath+filename+'.html')
    return fig